In [2]:
import numpy
import xarray
import matplotlib.pyplot as mp
import glob
import pandas
import cartopy

In [3]:
PRECT_root = '/ninad/NCAR_LENS/PRECT/'

Open RCP 8.5 runs for each ensemble member as multi-file dataset

In [7]:
members = ['{:03d}'.format(i) for i in list(range(1,36))+list(range(101,106))]

# open both 2006-2080 and 2080-2100

i=1
prect_rcp_mfds = xarray.open_mfdataset(PRECT_root + \
                                       'b.e11.BRCP85C5CNBDRD.f09_g16.' + \
                                       members[i] + '.cam.h1.PRECT.*.nc', chunks={'lat':10, 'lon':10}).sel(lat=slice(20,60),lon=slice(230,260))

prect_rcp = prect_rcp_mfds['PRECT']*86400.*1000.
lon = prect_rcp_mfds['lon']
lat = prect_rcp_mfds['lat']
time = prect_rcp_mfds['time'].values

Calculate rolling means and rolling sums on dask arrays

In [11]:
time_datetime = pandas.to_datetime(time)
time_djf_indices = [d.month in [12,1,2] for d in time_datetime]

Convert all values <0.1 to nan

In [16]:
#prect_rcp = prect_rcp.where(prect_rcp<0.1, drop=True).values
prect_rcp = prect_rcp.where(prect_rcp>0.1, drop=True)

In [17]:
prect_rcp.shape

(34675, 43, 25)

In [18]:
prect_rcp = prect_rcp.values

In [ ]:
prect_rcp[prect_rcp<0.1] = numpy.nan

In [72]:
prect_rcp_timemean = numpy.nanmean(prect_rcp, axis=0)
prect_djf_timemean = numpy.nanmean(prect_rcp[time_djf_indices,:,:], axis=0)

In [1]:
proj_map = cartopy.crs.PlateCarree()

fig = mp.figure(figsize=(10,10))

ax = fig.add_subplot(111, projection=proj_map)
ax.coastlines(resolution='50m')
plot = ax.contourf(prect_rcp_mfds['lon'].values, prect_rcp_mfds['lat'].values, prect_rcp_timemean)
mp.colorbar(plot)
#ax.set_extent([-130,-100,20,55])
ax.set_extent([-150,-80,10,70])

NameError: name 'cartopy' is not defined